In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [ ]:
# base_dir = 'datasets/UTKFace'
base_dir = 'mimiDataSet'

In [ ]:
images = os.listdir(base_dir)

In [ ]:
images[0].split('_')

In [ ]:
image_paths = []
age_label = []
gender_label = []

for filename in images:
    image_path = os.path.join(base_dir, filename)
    temp = filename.split('_')

    if str(temp[0]).isnumeric():
        image_paths.append(image_path)
        age_label.append(int(temp[0]))


In [ ]:
def make_dataset(images):
    features = []  # لیست برای نگه داشتن تصاویر پردازش‌شده

    for image in images:
        # مرحله 1: بارگذاری تصویر با اندازه مشخص و رنگ RGB
        img = load_img(
            image,
            target_size=(128, 128),
            color_mode='rgb',
            interpolation='lanczos',
        )

        # مرحله 2: تبدیل به آرایه numpy
        img_array = img_to_array(img)

        # مرحله 3: نرمال‌سازی (مقدار پیکسل‌ها بین 0 تا 1)
        img_array = img_array / 255.0

        # مرحله 4: اضافه کردن به لیست (بدون expand_dims)
        features.append(img_array)

    # مرحله 5: تبدیل لیست به آرایه numpy با شکل (N, 128, 128, 3)
    features = np.array(features)

    return features


In [ ]:
x = make_dataset(image_paths)
x.shape

In [ ]:
y = np.array(age_label)
y.shape

In [ ]:
model = Sequential()

model.add(Input(shape=(128, 128, 3)))  # تعریف ورودی به صورت جداگانه

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(units=132, activation='relu'))

model.add(Dense(1))  # خروجی نهایی

model.summary()


In [ ]:
model.compile(loss='mse', optimizer='adam' , metrics=['accuracy'] )

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42 )
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [ ]:
keras.saving.save_model(model, 'my_model.keras')

In [ ]:
import matplotlib.pyplot as plt

image_index = 4

# نمایش مقدار واقعی
print("🎯 Original Age:", y_test[image_index])

# پیش‌بینی سن
pred = model.predict(x_test[image_index].reshape(1, 128, 128, 3))

print("🤖 Predicted Age:", int(pred[0][0]))

# نمایش تصویر
plt.imshow(x_test[image_index])
plt.axis('off')
plt.title("Test Image")
plt.show()


In [ ]:
def make_test(images):
    features = []  # لیستی برای نگه داشتن همه‌ی تصاویر پردازش‌شده

    # Step 1: Load the image
    img = load_img(
        images,
        target_size=(128, 128),
        color_mode='rgb',
        interpolation='lanczos',
    )

    # Step 2: Convert to numpy array
    img_array = img_to_array(img)

    # Step 3: Normalize (اختیاری ولی معمول برای شبکه‌های عصبی)
    img_array = img_array / 255.0

    # Step 4: Expand dimensions (اگر می‌خوای با مدل Keras استفاده کنی)
    img_array = np.expand_dims(img_array, axis=0)

    # Step 5: Add to features
    features.append(img_array)

    # Optional: Merge all into a single array with shape (N, 128, 128, 3)
    features = np.vstack(features)

    return features

In [ ]:
test1 = make_test('test.jpg')